## 概要

- `gemini`を使用して、論文のアブストラクトの評価を行うスクリプト
- `gemini`の出力は JSON 形式に固定
- `rules`で評価指標を指定
- ``で用語等の定義を指定


In [15]:
# 初期設定
!pip install pandas
import pathlib
import textwrap
import google.generativeai as genai
import os
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown
import pandas as pd
import time
import json



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [16]:
# モデルのインスタンスを作成
model = genai.GenerativeModel(
    "gemini-1.5-flash",
    generation_config={
                        "response_mime_type": "application/json",
                        "temperature": 0,
                        "top_k": 1,
                    }
)

In [17]:
# geminiを使った回答の生成
def generate_response(model, prompt):
    response = model.generate_content(prompt)
    return response

def process_abstracts(model, abstracts, definition, instruction, rules, interval=5):
    results = []
    for abstract in abstracts:
        abstract_id = abstract["abstract_id"]
        content = abstract["content"]
        
        # プロンプトの作成
        prompt = create_prompt(content, definition, instruction, rules)
        
        # レスポンス生成
        response = generate_response(model, prompt)
        
        # 結果の保存
        results.append({
            "abstract_id": abstract_id,
            "response": response.text
        })
        
        # インターバルを挿入
        time.sleep(interval)
    
    return results

# responseカラムを解析してrule1, rule2, rule3に分解
def parse_response(response):
    if pd.isna(response):
        return {}
    try:
        # JSON形式をデコード
        parsed = json.loads(response)
        if isinstance(parsed, list):  # リストの場合
            if isinstance(parsed[0], dict) and "rules" in parsed[0]:
                rules = parsed[0]["rules"]
            else:
                rules = []
        elif isinstance(parsed, dict):  # 辞書の場合
            if "results" in parsed:  # "results"キーがある場合
                rules = parsed["results"][0].get("rules", [])
            else:  # "rules"キーが直接ある場合
                rules = parsed.get("rules", [])
        else:
            rules = []
        # ルールを辞書形式で返す
        return {f"rule{i+1}": rule for i, rule in enumerate(rules)}
    except json.JSONDecodeError:
        return {}

In [18]:
file_name = "Biochemistry_Molecular_Biology_low1000"
input_file = f"../data/csv/{file_name}.csv"
output_file = f"../data/results/{file_name}.csv"

# 読み込むデータの行数を指定
num_rows_to_read = 5

try:
    # df = pd.read_csv(input_file, encoding="utf-8", nrows=num_rows_to_read)
    df = pd.read_csv(input_file, encoding="utf-8")
    print("データの読み込みに成功しました。")
except Exception as e:
    print(f"エラーが発生しました: {e}")

データの読み込みに成功しました。


In [19]:
df["ID"] = df.index
cols = ["ID"] + [col for col in df.columns if col != "ID"]
df = df[cols]
df.head()

,ID,Publication Type,Authors,Title,Abstract,DOI
0,0,J,"Athare, SV; Gejji, SP",Regioselectivity in nonsymmetric methyl pentyl...,The present work illustrates regioselective bi...,10.1016/j.jmgm.2019.107960
1,1,J,"Brunetti, M; Mortola, JP",Hypoxic hypometabolism in chicken embryos: con...,"Postnatally, during hypoxia the decrease in ox...",10.1016/j.cbpa.2019.110578
2,2,J,"Du, ZF; Qu, Y; Farrell, NP",Intramolecular platinum migration on a peptide...,We report the migration of platinum ligand uni...,10.1016/j.jinorgbio.2019.110858
3,3,B,"Jahn, D; Geier, A",Transcriptional control of cells by vitamin D ...,NaN,10.1016/B978-0-12-811907-5.00030-0
4,4,J,"Pakravan, M; Shamsollahi, MB","Spatial and temporal joint, partially-joint an...",absectionBackground Three types of sources can...,10.1016/j.jneumeth.2019.108453


In [20]:
# 用語等の定義
definition = """
You will judge whether a given abstract meets the specified evaluation metrics based on the given criteria. The following is the prerequisite knowledge used when evaluating abstracts.

[## Prerequisites

**Proposed Method**: A newly conceived approach or method to solve a specific problem. This includes improvements to existing methods. It encompasses the steps and operations of the method.

- **Example 1 (New Algorithm):** Developed a new algorithm to solve traffic congestion by adjusting traffic volume and signal timing in real-time. This algorithm inputs past traffic data and current traffic conditions into a machine learning model to output optimal signal control patterns. The proposed method includes the algorithm design, creation of learning data, parameter tuning method, and the procedure for applying it to actual traffic signals.
- **Example 2 (Improvement of Existing Method):** Added a new preprocessing step to the existing image recognition model to improve accuracy. This preprocessing involves noise removal and contrast adjustment. The proposed method includes the specific steps of this preprocessing and how it can be integrated into image recognition models.

**Dataset**: A collection of data used for experiments or analysis.

- **Example 1 (Image Data):** A dataset consisting of 10,000 cat images and 10,000 dog images. Each image is labeled with breed, fur color, and posture.
- **Example 2 (Text Data):** A dataset of 1 million tweets. Each tweet includes posting date and time, user ID, content, and hashtags.

**Experimental Equipment**: Physical devices or facilities used in an experiment. Software is not included.

- **Example 1:** EEG measurement device, electrodes, amplifier
- **Example 2:** Beakers, flasks, electronic balance for observing chemical reactions

**Data Collection Method**: The method of obtaining data that constitutes the dataset.

- **Example 1 (Survey):** Created an online survey form and recruited participants through social media to collect responses.
- **Example 2 (Sensor Measurement):** Installed temperature and humidity sensors to automatically record data every hour.

**Evaluation Metrics**: Specific measures to assess the performance or effectiveness of the proposed method or experimental results.

- **Example 1 (Image Classification Accuracy):** Percentage of correctly classified images
- **Example 2 (Algorithm Execution Speed):** Time taken to complete a specific task

**Experimental Results/Findings**: Specific data obtained from the experiment and the insights derived from that data.

- **Example 1:** When using proposed method A, image classification accuracy was 95%. In contrast, existing method B achieved 80%. This result shows that proposed method A achieves higher classification accuracy than existing method B.
- **Example 2:** Mice administered drug X showed significantly larger tumor reduction compared to the control group (p < 0.05). This suggests that drug X has anti-tumor effects.

**Research Prospects**: Description of future research directions or potential applications of research findings.

- **Example 1:** Future research will need to verify the performance of the proposed method using larger datasets. We will also investigate potential applications in other fields.
- **Example 2:** It is expected that improving the algorithm developed in this research could enable the construction of faster and more energy-efficient systems.

**Research Background**: The context that gave rise to the research topic, explaining why the topic is important and why research is necessary. Describes social needs, academic challenges, or unresolved problems.

- **Example:** "In recent years, abnormal weather events have become frequent due to global warming. Particularly, damage from concentrated heavy rainfall has become serious, making the strengthening of disaster prevention measures urgent. However, existing disaster prevention systems have low prediction accuracy and difficulty in prompt response."

**Overview of Previous Research and Its Relevance**: Introduces existing research and clarifies its relationship with the current research. Explains the achievements and limitations of previous research, and the novelty and contribution of the current research.

- **Example:** "Conventional concentrated heavy rainfall prediction systems primarily relied on weather radar data. However, these radar data have low spatial resolution and cannot accurately capture localized heavy rainfall. This research proposes a method to improve prediction accuracy by integrating radar data with ground-based rainfall gauge data and information from social media."

**Research Objective**: Clearly describes what the research aims to achieve. Specifies concrete goals or challenges to be solved.

- **Example:** "The objective of this research is to improve the accuracy of concentrated heavy rainfall prediction by comprehensively utilizing diverse data sources."

**Result Analysis Method**: Explains the methods used to analyze experimental results. Describes specific method names and procedures such as statistical methods, machine learning techniques, or simulations.

- **Example:** "To evaluate prediction accuracy, RMSE (Root Mean Squared Error) was used. To compare the performance of the proposed and conventional methods, a t-test was used to verify statistical significance."

**Statistical Trends**: Describes the statistical characteristics of the data. Explains data distribution and trends using statistical measures such as mean, median, standard deviation, correlation coefficient.

- **Example:** "When using the proposed method, RMSE decreased by 20% compared to the conventional method. Additionally, the correlation coefficient between prediction results and actual measurements was 0.85, indicating a high correlation."

**Discussion**: Interprets experimental results, discussing their meaning and significance. Explores reasons for obtaining certain results, causes of unexpected findings, and research limitations.

- **Example:** "The proposed method showed improved prediction accuracy compared to conventional methods. This is likely because integrating diverse data sources allowed more accurate capture of localized heavy rainfall. However, this research focused only on urban areas, and the applicability to areas with complex topography remains a future challenge."

**Conclusion**: Summarizes the entire research and briefly describes the findings. Describes the degree of achieving research objectives, main results, and future prospects. "Code has been made publicly available." is not a conclusion.

- **Example:** "This research proposed a heavy rainfall prediction system that comprehensively utilizes diverse data sources and verified its effectiveness. Experimental results confirmed that the proposed method improves prediction accuracy compared to conventional methods. Future research will investigate applicability to areas with complex topography."

**Comparing Main Results with Traditional Thinking**: Comparing the achievements of the proposed method with existing research and methods, clearly demonstrating its advantages and novelty. Qualitative comparisons are also effective.

- **Example:** "Conventional heavy rainfall prediction systems, relying solely on weather radar data, found it difficult to predict localized heavy rainfall. In contrast, the proposed system, by utilizing rainfall gauge data and social media information in addition to radar data, improved localized heavy rainfall prediction accuracy by 20%. Moreover, while existing systems required hours for prediction, the proposed system enables real-time prediction."]
"""

# 指示部分
instruction = """
1. **Section Decomposition**:
Systematically divide the abstract into the following predefined sections:
    - Research Background
    - Prior Research
    - Research Purpose
    - Proposed Method
    - Dataset
    - Experimental Apparatus
    - Data Collection Method
    - Experimental Evaluation Metrics
    - Experimental Results
    - Fingings from the results
    - Result Analysis Method
    - Statistical Trends of Results
    - Quantitative Results
    - Qualitative Results
    - Discussion
    - Conclusion
    - Research Prospects
2. **Evaluation Format**:
Provide a JSON response that evaluates whether the abstract meets specific criteria for each section.
3. **Evaluation Rules**:
    - If a section is not present in the abstract, represent it as an empty list.
    - For evaluation metrics sections:
        - Answer with 'no' for whether something is written or not
        - Answer with 'happy' for questions related to it
        - For example, if the discussion is not clearly written, 'discussion is written' would be 'no', while 'discussion is written in present tense' would be 'happy'.

Use this JSON schema:

**JSON schema**:
{
    "results": [
        {
            "rules": ["yes" or "no", ...]
        }
    ]
}

**Example Response**:
{
    "results": [
        {
            "rules": ["yes", "no", "yes"]
        }
    ]
}
"""

# 評価指標
rules = """
Rules:
1. Research background is written
2. Research background is shown in one sentence
3. Research background is shown in two sentences
4. Overview of previous research and its relevance is written
5. Research objective is written
6. Research objective is in the first sentence of the abstract
7. Research objective is within the first three sentences of the abstract
8. Research objective is written in present tense
9. Research objective is written in past tense
10. The necessity of achieving the research objective is written
11. Proposed research method is written
12. Proposed research method is written in past tense
13. Explanation of dataset content exists
14. Experimental equipment is explained
15. Data collection method is written
16. Experimental evaluation metrics are written
17. Experimental results are written
18. Findings from the results are written
19. Experimental results and findings are in present tense
20. Experimental results and findings are in past tense
21. Result analysis method is written
22. Statistical trends are written
23. Quantitative results are written
24. Qualitative results are written
25. Discussion is written
26. Conclusion is written
27. Discussion is in present tense
28. Conclusion is in present tense
29. Main results are explained while comparing with traditional thinking
30. Research prospects are written in the last 1-2 sentences
31. Abbreviations are included (except when first explained)
32. Written in active voice
33. Use of I or We as the subject
"""

# プロンプトの作成関数
def create_prompt(abstract, definition, instruction, rules):
    return f"""
    {instruction}
    Abstract: {abstract}
    {definition}
    {rules}
    """
    


In [21]:
abstracts = [
    {"abstract_id": row["ID"], "content": row["Abstract"]}
    for _, row in df.dropna(subset=["Abstract"]).iterrows()
]

In [22]:
abstracts

[{'abstract_id': 0,
  'content': 'The present work illustrates regioselective binding of nonsymmetric axle BuX (X = F, Cl, Br, CN) and 5-bromovaleronitrile (BVN) to the non-symmetric methyl pentyl pillar[5]arene (MPP5). Theoretical calculations reveal that the guest encapsulation within MPP5 is spontaneous and the conformer showing X weakly bound to pentyl rim of MPP5 is favoured over its other conformer wherein it interacts with methyl rim of the host. The noncovalent interactions namely C-H-pi, C-H-X and H\\-H prevail over C- H center dot center dot center dot O hydrogen bonding in the complexes of MPP5. The manifestations of these to vibrational spectra obtained from the present theory are discussed. The strength of host-guest binding further is shown to correlate well with weakening of the C-X bond through natural bond orbital analyses. (C) 2019 Elsevier Inc. All rights reserved.'},
 {'abstract_id': 1,
  'content': 'Postnatally, during hypoxia the decrease in oxygen consumption ((V

In [23]:
%%time
raw_responses = process_abstracts(model, abstracts, definition, instruction, rules)

RetryError: Timeout of 600.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNAVAILABLE: ipv4:172.217.175.42:443: Failed to connect to remote host: Timeout occurred: FD shutdown

In [24]:
raw_responses

[{'abstract_id': 0,
  'response': '{"results": [{"rules": ["no", "happy", "happy", "no", "no", "no", "no", "happy", "happy", "no", "yes", "yes", "no", "no", "no", "no", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "yes", "no", "yes", "yes", "yes"]}]}'},
 {'abstract_id': 1,
  'response': '{"results": [{"rules": ["no", "no", "no", "no", "no", "no", "no", "happy", "no", "no", "yes", "yes", "no", "no", "yes", "no", "yes", "yes", "yes", "no", "no", "no", "yes", "yes", "yes", "yes", "yes", "happy", "happy", "no", "yes", "yes", "yes"]}]}'},
 {'abstract_id': 2,
  'response': '{"results": [{"rules": ["no", "happy", "happy", "no", "no", "happy", "happy", "happy", "no", "no", "yes", "yes", "no", "no", "yes", "no", "yes", "yes", "yes", "yes", "no", "no", "yes", "yes", "yes", "yes", "yes", "happy", "happy", "no", "no", "yes", "yes"]}]}'},
 {'abstract_id': 4,
  'response': '{"results": [{"rules": ["yes", "no", "no", "yes", "yes", "no", "no", "no", "yes", "no", 

In [25]:
# DataFrameに変換
results_df = pd.DataFrame(raw_responses)

# `response`をパースして新しいカラムを作成
rules_df = results_df["response"].apply(parse_response).apply(pd.Series)

# `abstract_id`にルールを結合
results_df = pd.concat([results_df, rules_df], axis=1).drop(columns=["response"])

# 元のDataFrameと評価結果を結合
merged_df = df.merge(results_df, left_on="ID", right_on="abstract_id", how="left").drop(columns=["abstract_id"])


In [26]:
merged_df.head()

,ID,Publication Type,Authors,Title,Abstract,DOI,rule1,rule2,rule3,rule4,...,rule24,rule25,rule26,rule27,rule28,rule29,rule30,rule31,rule32,rule33
0,0,J,"Athare, SV; Gejji, SP",Regioselectivity in nonsymmetric methyl pentyl...,The present work illustrates regioselective bi...,10.1016/j.jmgm.2019.107960,no,happy,happy,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,yes
1,1,J,"Brunetti, M; Mortola, JP",Hypoxic hypometabolism in chicken embryos: con...,"Postnatally, during hypoxia the decrease in ox...",10.1016/j.cbpa.2019.110578,no,no,no,no,...,yes,yes,yes,yes,happy,happy,no,yes,yes,yes
2,2,J,"Du, ZF; Qu, Y; Farrell, NP",Intramolecular platinum migration on a peptide...,We report the migration of platinum ligand uni...,10.1016/j.jinorgbio.2019.110858,no,happy,happy,no,...,yes,yes,yes,yes,happy,happy,no,no,yes,yes
3,3,B,"Jahn, D; Geier, A",Transcriptional control of cells by vitamin D ...,NaN,10.1016/B978-0-12-811907-5.00030-0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,J,"Pakravan, M; Shamsollahi, MB","Spatial and temporal joint, partially-joint an...",absectionBackground Three types of sources can...,10.1016/j.jneumeth.2019.108453,yes,no,no,yes,...,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes


In [27]:
# 除外したいカラムを指定
columns_to_exclude = ["Publication Type", "Authors", "Title", "DOI"]

# 指定したカラム以外を表示
filtered_df = merged_df.drop(columns=columns_to_exclude).head()


In [28]:
filtered_df.to_csv(output_file, index=False, encoding="utf-8")